# Feedback Prize Best score 0.6

In [1]:
import pandas as pd
import os
path = ''

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv(path+"sample_submission.csv")

In [4]:
df

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [5]:
df = pd.read_csv(path+"train.csv")

In [9]:
len(all_types)
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
from torch.utils.data import Dataset
import numpy as np
import torch
import nltk
import re

class GrammerDataset(Dataset):
    pos_tag_vocab = ['CC',
            'WRB',
            'EX',
            'MD',
            'VBN',
            'VBD',
            'NNS',
            'RBR',
            'VBZ',
            'PRP$',
            'VB',
            'RP',
            'WP',
            'VBP',
            'JJR',
            'VBG',
            'PDT',
            'JJ',
            'JJS',
            'WDT',
            'IN',
            'DT',
            'RB',
            'NN',
            'PRP',
            'TO']

    def __init__(self, data):
        '''
        Dataset object for base model
        :param data:
        '''
        self.text,self.text_length,self.pos_tag = self.clean_text(data['full_text'])
        self.cohesion = np.array(self.feature_transformation(data['cohesion']))
        self.syntax = np.array(self.feature_transformation(data['syntax']))
        self.vocab = np.array(self.feature_transformation((data['vocabulary'])))
        self.phraseology = np.array(self.feature_transformation(data['phraseology']))
        self.grammer = np.array(self.feature_transformation(data['grammar']))
        self.conventions = np.array(self.feature_transformation(data['conventions']))
        

    def __len__(self):
        return len(self.text)
    
    def feature_transformation(self, vals):
        self.mapping = {1:0,1.5:1,2:2,2.5:3,3:4,3.5:5,4:6,4.5:7,5:8}
        newl = []
        for v in vals:
            newl.append(self.mapping[v])
        return newl
    def count_pos_tag(self,text):
        tag_dict = {}
        pos = nltk.pos_tag(text)
        tag_types = list(set([item[1] for item in pos]))
        for tag in tag_types:
            tag_dict[tag] = sum([item[1]==tag for item in pos])
        return tag_dict
    

    def clean_text(self,all_text):
        newl = []
        newlength = []
        pos_tag = []
        for text in all_text:
            text = text.replace("\n"," ").lower()
            text = text.strip()
            newl.append(text)
            newlength.append([len(t) for t in text.split()])
            pos_sentence = self.count_pos_tag(text.split())
            hot_vector_pos = []
            for po in self.pos_tag_vocab:
                if po in pos_sentence.keys():
                    hot_vector_pos.append(pos_sentence[po])
                else:
                    hot_vector_pos.append(0)
            pos_tag.append(hot_vector_pos)
            
            
        return np.array(newl),np.array(newlength),np.array(pos_tag)
    def __getitem__(self, idx):
        return self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx],self.text_length[idx],self.pos_tag[idx]

In [11]:
train=df.sample(frac=0.95,random_state=155) #random state is a seed value
test=df.drop(train.index)
train_dataset = GrammerDataset(train)
valid_dataset = GrammerDataset(test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [12]:
from torchtext.data.utils import(
    get_tokenizer,
    ngrams_iterator,
)
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
ngrams = 2
def yield_tokens(data_iter, ngrams):
    for text in data_iter:
        yield iter(tokenizer(text))

myvocab = build_vocab_from_iterator(yield_tokens(train_dataset.text, ngrams), specials=["<unk>"])
myvocab.set_default_index(myvocab["<unk>"])
device = 'cuda'
def text_pipeline(x,padding=1024): 
    res = myvocab(list((tokenizer(x))))
    if len(res) < padding:
        chars_to_add = int(padding-len(res))
        for i in range(chars_to_add):
            res.append(myvocab['<pad>'])
    else:
        res = res[:padding]
    return res
# self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
def collate_batch(batch):
    text_list, cohesion, syntax,vocab,phraseology,grammer,conventions,text_length,pos_tag = [], [],[],[],[],[],[],[],[]
    for data in batch:
        processed_text = text_pipeline(data[0],1024)
        text_list.append(processed_text)
        text_length.append(len(processed_text))
        cohesion.append(data[1])
        syntax.append(data[2])
        vocab.append(data[3])
        phraseology.append(data[4])
        grammer.append(data[5])
        conventions.append(data[6])
        pos_tag.append(data[8])
        
        
    cohesion = torch.tensor(cohesion, dtype=torch.int64)
    syntax = torch.tensor(syntax, dtype=torch.int64)
    vocab = torch.tensor(vocab, dtype=torch.int64)
    phraseology = torch.tensor(phraseology, dtype=torch.int64)
    grammer = torch.tensor(grammer, dtype=torch.int64)
    conventions = torch.tensor(conventions, dtype=torch.int64)
    pos_tag_vals = torch.tensor(pos_tag, dtype=torch.int64)
    text_length = torch.tensor(text_length, dtype=torch.int64)
    text_list = torch.tensor(text_list, dtype=torch.int64)
    return text_list.to(device), cohesion.to(device), syntax.to(device),vocab.to(device), phraseology.to(device),grammer.to(device), conventions.to(device),text_length.to(device),pos_tag_vals.to(device)

In [13]:
import torch.nn as nn
import torch
from transformers import DebertaTokenizer, DebertaModel
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss


class MCRMSELoss(nn.Module):
    def __init__(self, num_scored=3):
        super().__init__()
        self.rmse = RMSELoss()
        self.num_scored = num_scored

    def forward(self, yhat, y):
        score = 0
        for i in range(self.num_scored):
            score += self.rmse(yhat[:, :, i], y[:, :, i]) / self.num_scored

        return score
    
    
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, bidirectional, hidden_dim, num_layers, output_dim, dropout=0.3, pad_idx=0,
                 fc_hidden2=64, fc_hidden1=256):
        super().__init__()
        self.hidden_size = fc_hidden2
        self.input_dim = input_dim
        self.bid = bidirectional
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.dropout_rate = dropout 
        self.fc_hidden = fc_hidden1


        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)

        # embedded to cuda if available
        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=num_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)
        

        #tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
        #model = DebertaModel.from_pretrained('microsoft/deberta-base', return_dict=True)

#         inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
#         outputs = model(**inputs,return_dict=True)

        self.num_neurons = (hidden_dim * num_layers) + 26 
        
        self.labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
        self.featurs_nn = []
        for lab in self.labels:
            fc = [nn.Linear(self.num_neurons, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout), nn.Linear(self.fc_hidden, self.fc_hidden), nn.ReLU(inplace=True), nn.Dropout(dropout),
                  nn.Linear(self.fc_hidden, self.hidden_size), nn.ReLU(inplace=True),nn.Linear(self.hidden_size, output_dim)]
            fc = nn.Sequential(*fc)
            self.featurs_nn.append(fc)
            
        self.featurs_nn = nn.Sequential(*self.featurs_nn)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text,text_length,pos_tag):
        embedded = self.dropout(self.embedding(text))
        text_length = torch.clamp(text_length, min=1)
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu(),batch_first=True,enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        res = torch.cat((hidden,pos_tag),dim=1)
        output_dict = {}
        for f_nn,feature in zip(self.featurs_nn,self.labels):
            newh = res.clone()
            out = f_nn(newh)
            output_dict[feature] = out
        #hidden = self.fc(hidden)
        return output_dict

def save_model(model, path):
    global myvocab
    cfg = model.state_dict()
    cfg['input_dim'] = model.input_dim
    cfg['embedding_dim'] = model.embedding_dim
    cfg['bidirectional'] = model.bid
    cfg['hidden_dim'] = model.hidden_dim
    cfg['num_layers'] = model.num_layers
    cfg['output_dim'] = model.output_dim
    cfg['dropout'] = model.dropout_rate
    torch.save(cfg, path)
    torch.save(myvocab,path.replace(".pth","")+'_vocab.pth')
    
def load_model(path):
    keys_to_remove = ['input_dim', 'embedding_dim', 'bidirectional', 'hidden_dim', 'num_layers', 'output_dim','dropout']
    cfg = torch.load(path)
    model = RNN_CNN(cfg['input_dim'],cfg['embedding_dim'],cfg['bidirectional'],cfg['hidden_dim'],cfg['num_layers'],cfg['output_dim'],cfg['dropout'])
    
    for key in keys_to_remove:
        cfg.pop(key)
    model.load_state_dict(cfg)
    model.eval()
    return model

def load_vocab(path):
    vocab = torch.load(path)
    return vocab
    

In [15]:
vocab_size = len(myvocab)
batch_size = 64
bidirectional = True
emb_dim = 300
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_batch)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_batch)

model = RNN(vocab_size, emb_dim,bidirectional,hidden_dim=256,num_layers=2,output_dim=9)
model.to(device)

RNN(
  (embedding): Embedding(21426, 300, padding_idx=0)
  (rnn): LSTM(300, 256, num_layers=2, dropout=0.3, bidirectional=True)
  (featurs_nn): Sequential(
    (0): Sequential(
      (0): Linear(in_features=538, out_features=256, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=256, out_features=256, bias=True)
      (4): ReLU(inplace=True)
      (5): Dropout(p=0.3, inplace=False)
      (6): Linear(in_features=256, out_features=64, bias=True)
      (7): ReLU(inplace=True)
      (8): Linear(in_features=64, out_features=9, bias=True)
    )
    (1): Sequential(
      (0): Linear(in_features=538, out_features=256, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=256, out_features=256, bias=True)
      (4): ReLU(inplace=True)
      (5): Dropout(p=0.3, inplace=False)
      (6): Linear(in_features=256, out_features=64, bias=True)
      (7): ReLU(inplace=True)
      (8)

In [16]:
def transform_labels(y_vals):
    mapping = {0: 1, 1: 1.5, 2: 2, 3: 2.5, 4: 3, 5: 3.5, 6: 4, 7: 4.5, 8: 5}
    if isinstance(y_vals, torch.Tensor):
        if len(y_vals) > 0:
            y_vals = y_vals.tolist()
            newl = []
            for i in y_vals:
                newl.append(mapping[i])
            return newl
        return mapping[y_vals.item()]
    elif isinstance(y_vals,int):
         return mapping[y_vals]
    else:
        if len(y_vals) > 0:
            y_vals = y_vals.tolist()
            newl = []
            for i in y_vals:
                newl.append(mapping[i])
            return newl
def compute_acc(predicted,gt):
    total_true = sum([predicted[i]==gt[i] for i in range(len(gt))])
    return total_true/len(gt), total_true
def compare_tensor_acc(preds, gt):
    preds = torch.softmax(preds, dim=1)
    proba, class_label = preds.max(dim=1)
    predict_total = [(transform_labels(cl), pr) for cl,pr in zip(class_label.tolist(),proba.tolist())]
    predict_only = [pt[0] for pt in predict_total]
    gra_acc,true = compute_acc(predict_only,gt.tolist())
    return predict_only,true

import math
def mcrmse_score(labels,y_size=6):
    #labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
    multiplier = (1/y_size)
    loss_sum = 0
    i = 0
    for item in labels:
        pred_vector = item[0]
        gt_vector = item[1]
        loss_sum += math.sqrt((1/len(pred_vector)) * sum([(pred_vector[i]-gt_vector[i])**2 for i in range(len(pred_vector))]))
    
    return loss_sum*multiplier

In [27]:
torch.set_default_dtype(torch.float64)

In [17]:
loss_func = nn.CrossEntropyLoss()
lr = 0.001
max_epochs = 200
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
best_loss = 9999
test_score = 0.35
save_path = 'test/best_model_v5.pth'
for epoch in range(max_epochs):
    all_losses = [] 
    print("Epoch {}".format(epoch))
    coh_train_correct = 0
    syn_train_correct = 0
    voc_train_correct = 0
    phr_train_correct = 0
    gr_train_correct = 0
    con_train_correct = 0
    train_samples = 0
    mcrmse_s = 0

    for batch_idx, batch_data in enumerate(train_loader):
        # self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
        text,cohesion,syntax,vocab,phraseology,grammer,conventions,text_length,pos_tag = batch_data
        outputs = model(text,text_length,pos_tag)


        # loss
        loss = 0
        loss += loss_func(outputs['cohesion'], cohesion)
        loss += loss_func(outputs['syntax'], syntax)
        loss += loss_func(outputs['vocabulary'], vocab)
        loss += loss_func(outputs['phraseology'], phraseology)
        loss += loss_func(outputs['grammar'], grammer)
        loss += loss_func(outputs['conventions'], conventions)
        
        
        if batch_idx % 100 == 0:
            print("Loss: {}".format(loss.item()))

        coh_acc,coh_true = compare_tensor_acc(outputs['cohesion'],cohesion)
        syn_acc,syn_true = compare_tensor_acc(outputs['syntax'],syntax)
        voc_acc,voc_true = compare_tensor_acc(outputs['vocabulary'],vocab)
        phr_acc,phr_true = compare_tensor_acc(outputs['phraseology'],phraseology)
        gr_acc,gr_true = compare_tensor_acc(outputs['grammar'],grammer)
        con_acc,con_true = compare_tensor_acc(outputs['conventions'],conventions)


        scores = []
        scores.append([coh_acc,cohesion])
        scores.append([syn_acc,syntax])
        scores.append([voc_acc,vocab])
        scores.append([phr_acc,phraseology])
        scores.append([gr_acc,grammer])
        scores.append([con_acc,conventions])

        batch_score = mcrmse_score(scores)
        mcrmse_s += batch_score
#         if batch_idx % 10 == 0:
#             print("MCRmse batch score: ", batch_score)

        all_losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        coh_train_correct += coh_true
        syn_train_correct += syn_true
        voc_train_correct += voc_true
        phr_train_correct += phr_true
        gr_train_correct += gr_true
        con_train_correct += con_true
        train_samples+= len(outputs['cohesion'].tolist())
    def avg(lst):
        return sum(lst) / len(lst)
    
    # calculating mcrmse metric
    mcrmse_epoch_score = mcrmse_s/len(train_loader)
    if mcrmse_epoch_score < best_loss:
        best_loss = mcrmse_epoch_score
        save_model(model,save_path)
        print("New model saved! better average RMCMSE score ", best_loss)
        
    if epoch % 10 == 0:
        print("Average loss: ", avg(all_losses))
        print("MCRmse average score: ", mcrmse_s/len(train_loader))
        print("Training accuracy!")
        print("Cohesion: ", coh_train_correct/train_samples, " Syntax: ", syn_train_correct/train_samples, " Vocab: ", voc_train_correct/train_samples)
        print("Phraseology: ", phr_train_correct/train_samples, " Grammar: ", gr_train_correct/train_samples, " Conventions: ", con_train_correct/train_samples)
    
    ## validation
    coh_correct = 0
    syntax_correct = 0
    vocab_correct = 0
    phraseology_correct = 0
    grammar_correct = 0
    conventions_correct = 0
    total_samples = 0

    coh_ans = []
    syntax_ans = []
    voc_ans = []
    phr_ans = []
    gra_ans = []
    conv_ans = []
    text_original = []
    valid_score = 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(valid_loader):
            text,cohesion,syntax,vocab,phraseology,grammer,conventions,text_length,pos_tag = batch_data
            outputs = model(text,text_length,pos_tag)
            coh_acc,coh_true = compare_tensor_acc(outputs['cohesion'],cohesion)
            syn_acc,syn_true = compare_tensor_acc(outputs['syntax'],syntax)
            voc_acc,voc_true = compare_tensor_acc(outputs['vocabulary'],vocab)
            phr_acc,phr_true = compare_tensor_acc(outputs['phraseology'],phraseology)
            gr_acc,gr_true = compare_tensor_acc(outputs['grammar'],grammer)
            con_acc,con_true = compare_tensor_acc(outputs['conventions'],conventions)


            scores = []
            scores.append([coh_acc,cohesion])
            scores.append([syn_acc,syntax])
            scores.append([voc_acc,vocab])
            scores.append([phr_acc,phraseology])
            scores.append([gr_acc,grammer])
            scores.append([con_acc,conventions])

            batch_score = mcrmse_score(scores)
            valid_score += batch_score


    print("Valdiation Avg Score: " , np.mean(valid_score))



Epoch 0
Loss: 13.35218620300293


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352464346/work/torch/csrc/utils/tensor_new.cpp:201.)


New model saved! better average RMCMSE score  1.7371624703125117
Average loss:  10.470125667119431
MCRmse average score:  1.7371624703125117
Training accuracy!
Cohesion:  0.10901749663526245  Syntax:  0.14212651413189772  Vocab:  0.12839838492597577
Phraseology:  0.12355316285329744  Grammar:  0.1028263795423957  Conventions:  0.1440107671601615
Valdiation Avg Score:  6.935617606861866
Epoch 1
Loss: 10.363153457641602
New model saved! better average RMCMSE score  1.6920933347104405
Valdiation Avg Score:  6.817420096488002
Epoch 2
Loss: 9.287731170654297
New model saved! better average RMCMSE score  1.6675317832976049
Valdiation Avg Score:  7.436793353216184
Epoch 3
Loss: 9.721858978271484
New model saved! better average RMCMSE score  1.6540832042866145
Valdiation Avg Score:  6.6048067458980135
Epoch 4
Loss: 9.190184593200684
New model saved! better average RMCMSE score  1.6482430332750444
Valdiation Avg Score:  6.040112520647576
Epoch 5
Loss: 8.380167961120605
New model saved! better a

Valdiation Avg Score:  6.440729884636751
Epoch 59
Loss: 2.9732625484466553
Valdiation Avg Score:  6.303834462203202
Epoch 60
Loss: 3.4936985969543457
New model saved! better average RMCMSE score  1.3295107728501832
Average loss:  3.1511811967623435
MCRmse average score:  1.3295107728501832
Training accuracy!
Cohesion:  0.08209959623149395  Syntax:  0.12139973082099596  Vocab:  0.04495289367429341
Phraseology:  0.09986541049798116  Grammar:  0.13566621803499326  Conventions:  0.12032301480484522
Valdiation Avg Score:  7.644958763080764
Epoch 61
Loss: 3.0555410385131836
New model saved! better average RMCMSE score  1.3229840194514848
Valdiation Avg Score:  5.7140864077724185
Epoch 62
Loss: 2.8617007732391357
Valdiation Avg Score:  5.974027154748553
Epoch 63
Loss: 3.3230271339416504
Valdiation Avg Score:  6.1893547968974705
Epoch 64
Loss: 2.904573678970337
Valdiation Avg Score:  7.4673883619246935
Epoch 65
Loss: 2.906820058822632
Valdiation Avg Score:  7.272625740546034
Epoch 66
Loss: 2.1

Valdiation Avg Score:  6.347886242466917
Epoch 128
Loss: 0.9425005316734314
Valdiation Avg Score:  6.982790575340987
Epoch 129
Loss: 1.028239130973816
Valdiation Avg Score:  7.303817212354106
Epoch 130
Loss: 0.8157036304473877
Average loss:  1.3226053159115678
MCRmse average score:  1.30447616291813
Training accuracy!
Cohesion:  0.08398384925975774  Syntax:  0.10874831763122476  Vocab:  0.03526244952893674
Phraseology:  0.09259757738896367  Grammar:  0.1440107671601615  Conventions:  0.11197846567967698
Valdiation Avg Score:  6.469885425960305
Epoch 131
Loss: 0.9140227437019348
Valdiation Avg Score:  6.892645710341861
Epoch 132
Loss: 0.879353404045105
Valdiation Avg Score:  6.727481706852082
Epoch 133
Loss: 0.9392303228378296
New model saved! better average RMCMSE score  1.2933026544958375
Valdiation Avg Score:  6.658221747198617
Epoch 134
Loss: 1.4672948122024536
Valdiation Avg Score:  6.925335144958586
Epoch 135
Loss: 1.300905466079712
Valdiation Avg Score:  6.680457871390596
Epoch 1

In [18]:
from torch.utils.data import Dataset
import numpy as np
import torch
class GrammerTestDataset(Dataset):
    pos_tag_vocab = ['CC',
            'WRB',
            'EX',
            'MD',
            'VBN',
            'VBD',
            'NNS',
            'RBR',
            'VBZ',
            'PRP$',
            'VB',
            'RP',
            'WP',
            'VBP',
            'JJR',
            'VBG',
            'PDT',
            'JJ',
            'JJS',
            'WDT',
            'IN',
            'DT',
            'RB',
            'NN',
            'PRP',
            'TO']
    def __init__(self, data):
        '''
        Dataset object for base model
        :param data:
        '''
        self.text,self.text_length,self.pos_tag = self.clean_text(data['full_text'])
        self.text_labels = data['text_id'].to_numpy()
        

    def __len__(self):
        return len(self.text)
    
    def feature_transformation(self, vals):
        self.mapping = {1:0,1.5:1,2:2,2.5:3,3:4,3.5:5,4:6,4.5:7,5:8}
        newl = []
        for v in vals:
            newl.append(self.mapping[v])
        return newl
    
    def count_pos_tag(self,text):
        tag_dict = {}
        pos = nltk.pos_tag(text)
        tag_types = list(set([item[1] for item in pos]))
        for tag in tag_types:
            tag_dict[tag] = sum([item[1]==tag for item in pos])
        return tag_dict

    def clean_text(self,all_text):
        newl = []
        newlength = []
        pos_tag = []
        for text in all_text:
            text = text.replace("\n"," ").lower()
            text = text.strip()
            newl.append(text)
            newlength.append([len(t) for t in text.split()])
            pos_sentence = self.count_pos_tag(text.split())
            hot_vector_pos = []
            for po in self.pos_tag_vocab:
                if po in pos_sentence.keys():
                    hot_vector_pos.append(pos_sentence[po])
                else:
                    hot_vector_pos.append(0)
            pos_tag.append(hot_vector_pos)
        return np.array(newl),np.array(newlength),np.array(pos_tag)
    def __getitem__(self, idx):
        return self.text[idx],self.text_length[idx],self.pos_tag[idx],self.text_labels[idx]

In [107]:
testdf

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [19]:
testdf = pd.read_csv(path+"test.csv")
testdata = GrammerTestDataset(testdf)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [20]:
# self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
def collate_test_batch(batch):
    text_list,text_length,pos_tag,text_id = [], [],[],[]
    for data in batch:
        processed_text = text_pipeline(data[0],1024)
        text_list.append(processed_text)
        text_length.append(len(processed_text))
        pos_tag.append(data[2])
        text_id.append(data[3])
        
        
        
    text_length = torch.tensor(text_length, dtype=torch.int64)
    text_list = torch.tensor(text_list, dtype=torch.int64)
    pos_tag = torch.tensor(pos_tag, dtype=torch.int64)
    return text_list.to(device), text_length.to(device),pos_tag.to(device),text_id

In [21]:

def test(model,loader,return_preds=False):
    coh_correct = 0
    syntax_correct = 0
    vocab_correct = 0
    phraseology_correct = 0
    grammar_correct = 0
    conventions_correct = 0
    total_samples = 0
    
    coh_ans = []
    syntax_ans = []
    voc_ans = []
    phr_ans = []
    gra_ans = []
    conv_ans = []
    text_original = []
    
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(loader):
            # self.text[idx], self.cohesion[idx],self.syntax[idx],self.vocab[idx],self.phraseology[idx],self.grammer[idx],self.conventions[idx]
            text = batch_data[0]
            text_length = batch_data[1]
            pos_tag = batch_data[2]
            text_id = batch_data[3]
            outputs = model(text,text_length,pos_tag)

            labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
            
            
            # cohesion 
            #a, predicted = torch.max(outputs['cohesion'],1)
            preds = torch.softmax(outputs['cohesion'], dim=1)
            proba, class_label = preds.max(dim=1)
            [coh_ans.append((transform_labels(class_label), proba.item())) for res in preds]
            
             # syntax 
            preds = torch.softmax(outputs['syntax'], dim=1)
            proba, class_label = preds.max(dim=1)
            [syntax_ans.append((transform_labels(class_label), proba.item())) for res in preds]
            
            # vocab 
            preds = torch.softmax(outputs['vocabulary'], dim=1)
            proba, class_label = preds.max(dim=1)
            [voc_ans.append((transform_labels(class_label), proba.item())) for res in preds]

             # phraseology    
            preds = torch.softmax(outputs['phraseology'], dim=1)
            proba, class_label = preds.max(dim=1)
            [phr_ans.append((transform_labels(class_label), proba.item())) for res in preds]
            # grammar 
            preds = torch.softmax(outputs['grammar'], dim=1)
            proba, class_label = preds.max(dim=1)
            [gra_ans.append((transform_labels(class_label), proba.item())) for res in preds]
            

            # conventions 
            preds = torch.softmax(outputs['conventions'], dim=1)
            proba, class_label = preds.max(dim=1)
            [conv_ans.append((transform_labels(class_label), proba.item())) for res in preds]

            total_samples+= len((outputs['conventions']))
            
            text_original.append(text_id)           
    
    
    coh_correct /= total_samples
    syntax_correct /= total_samples
    vocab_correct /= total_samples
    phraseology_correct /= total_samples
    grammar_correct /= total_samples
    conventions_correct /= total_samples
    
#     print("Accuracy Cohesion: ", coh_correct)
#     print("Accuracy Syntax: ", syntax_correct)
#     print("Accuracy Vocab: ", vocab_correct)
#     print("Accuracy Phraselogy: ", phraseology_correct)
#     print("Accuracy Grammer: ", grammar_correct)
#     print("Accuracy Convetions: ", conventions_correct)
    
    if return_preds:
        return text_original, coh_ans,syntax_ans,voc_ans,phr_ans,gra_ans,conv_ans
 
        
        
    return coh_correct,syntax_correct,vocab_correct,phraseology_correct,grammar_correct,conventions_correct
    
    
                            
        


In [24]:
testloader = DataLoader(testdata, batch_size=1, shuffle=False,collate_fn=collate_test_batch)
newmodel = load_model(save_path)
newmodel.to(device)
all_res = test(newmodel,testloader,return_preds=True)

In [25]:
all_res

([['0000C359D63E'], ['000BAD50D026'], ['00367BB2546B']],
 [([3], 0.8307837247848511),
  ([3], 0.6173878312110901),
  ([3], 0.9640266299247742)],
 [([3], 0.9900375604629517),
  ([2.5], 0.9973722696304321),
  ([3], 0.9915764927864075)],
 [([2.5], 0.29021111130714417),
  ([3], 0.8731154799461365),
  ([3.5], 0.654915452003479)],
 [([3], 0.776765763759613),
  ([2.5], 0.9945618510246277),
  ([3], 0.7955027222633362)],
 [([3], 0.5797446966171265),
  ([2], 0.8157117962837219),
  ([3.5], 0.8865168690681458)],
 [([3], 0.7724350094795227),
  ([3], 0.9311811923980713),
  ([3], 0.9867558479309082)])

In [26]:
test_pred = pd.DataFrame()
test_pred['text_id'] = [item[0] for item in all_res[0]]
test_pred['cohesion'] = [item[0][0] for item in all_res[1]]
test_pred['syntax'] = [item[0][0] for item in all_res[2]]
test_pred['vocabulary'] = [item[0][0] for item in all_res[3]]
test_pred['phraseology'] = [item[0][0] for item in all_res[4]]
test_pred['grammar'] = [item[0][0] for item in all_res[5]]
test_pred['conventions'] = [item[0][0] for item in all_res[6]]

In [27]:
testtt = pd.read_csv(path+"sample_submission.csv")

In [28]:
testtt

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [29]:
test_pred

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3,3.0,2.5,3.0,3.0,3
1,000BAD50D026,3,2.5,3.0,2.5,2.0,3
2,00367BB2546B,3,3.0,3.5,3.0,3.5,3


In [30]:
import math
def mcrmse(predicted_df,gt_df,y_size=6):
    labels = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
    multiplier = (1/y_size)
    loss_sum = 0
    for lab in labels:
        pred_vector = predicted_df[lab].to_list()
        gt_vector = gt_df[lab].to_list()
        loss_sum += math.sqrt(1/len(pred_vector) * sum([(pred_vector[i]-gt_vector[i])**2 for i in range(len(pred_vector))]))
    
    return loss_sum*multiplier
    

In [31]:
mcrmse(testtt,test_pred)

0.27184929733689855